# EEP153 Project 2 - Team Wilbur-Atwater

### Table of Contents
I. [Question A](#QuestionA)<br>
> Ia. [Create the Dietary_Reference_Intake_Function](#Dietary_Reference_Intake_Function)<br>
    
II. [Linear Program Setup](#Linear_Program_Setup)<br>
> IIa. [Create bmin and bmax Dataframes](#Bmin_Bmax)<br>

> IIb. [Create diet_min and diet_max Dataframes](#Dietmin_Dietmax)<br>

> IIc. [Create df, Price, and FoodNutrients Dataframes](#df_Price_FoodNutrients)<br>
    >> ia. [Budget Friendly Diabetes Diet](#Budget1)<br>
    >> ia. [Dash Diet](#Dash1)<br>
    
> IId. [Create Amin, Amax, and A Dataframes](#Amin_Amax_A)<br>
    >> ia. [Budget Friendly Diabetes Diet](#Budget2)<br>
    >> ia. [Dash Diet](#Dash2)<br>
    
> IIe. [Create the b Dataframe](#b)<br>

> IIf. [Solving the Linear Program Problem](#Solved)<br>
    >> ia. [Budget Friendly Diabetes Diet](#Budget3)<br>
    >> ia. [Dash Diet](#Dash3)<br>
    
III. [Solve the Subsistence Problem](#Subsistence)<br>
> IIIa. [Create the Solve_Subsistence_Problem Function](#Subsistence_function)<br>

> IIIb. [Results For Target Group](#Results)<br>
    >> ia. [Budget Friendly Diabetes Diet](#Budget4)<br>
    >> ia. [Dash Diet](#Dash4)<br>
    
> IIIc. [Plotting Foods vs Prices](#Plotting)<br>
    >> ia. [Budget Friendly Diabetes Diet](#Budget5)<br>
    >> ia. [Dash Diet](#Dash5)<br>

## Question A <a id='QuestionA'></a>

### Install and import necessary libraries

In [174]:
!pip install -r requirements.txt

In [175]:
import fooddatacentral as fdc
import numpy as np
import pandas as pd
from eep153_tools import read_sheets
from scipy.optimize import linprog as lp

## Create the dietary_reference_intake_function <a id='Dietary_Reference_Intake_Function'></a>

### Read in the minimum diet requirements dataframe

Energy: Calories
Protein: Grams
Fiber: Grams
Folate DFE: Micrograms
Calcium Ca: Milligrams
Carbohydrate by difference: Grams
Iron Fe: Milligrams
Magnesium Mg: Milligrams
Niacin: Milligrams
Phosphorus P: Milligrams
Potassium K: Milligrams
Riboflavin: Milligrams
Thiamin: Milligrams
Vitamin A RAE: Micrograms
Vitamin B-12: Milligrams
Vitamin B-6: Micrograms
Vitamin C total ascorbic acid: Milligrams 
Vitamin E (alpha-tocopherol): Milligrams
Vitamin K (phylloquinone): Micrograms
Zinc Zn: Micrograms

In [176]:
min_diet_reqs_df = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
min_diet_reqs_df.head()

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


In [177]:
diet_age_range_df = min_diet_reqs_df[["F 19-30", "M 19-30", "F 31-50", "M 31-50", "F 51+", "M 51+"]]
diet_age_range_df.head()

,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,
Energy,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


### Scalars

Low in saturated and trans fats
Rich in potassium, calcium, magnesium, fiber, and protein
Lower in sodium

Carbs: 55%
Potassium: 4700 mg
Calcium: 1250 mg
Magnesium: 500 g

In [178]:
for i in range(len(diet_age_range_df.loc["Carbohydrate, by difference",:])):
    diet_age_range_df.loc["Carbohydrate, by difference",:][i] = (diet_age_range_df.loc["Energy",:][i] * .55)
               
diet_age_range_df.loc["Calcium, Ca",:] = 1200
diet_age_range_df.loc["Magnesium, Mg",:] = 500
diet_age_range_df.head()

,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,
Energy,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",1200.0,1200.0,1200.0,1200.0,1200.0,1200.0


### dietary_reference_intake_function 

In [179]:
def dietary_reference_intake_function(group, activity_level=None):
    
    sex_age_series = diet_age_range_df[group]
    
    if activity_level:
        if activity_level != sex_age_series[0]:
            scalar = activity_level / sex_age_series[0]
            sex_age_series = sex_age_series.multiply(scalar)
    
    return sex_age_series
                                                 
dietary_reference_intake_function('M 19-30')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1200.0
Carbohydrate, by difference       1320.0
Iron, Fe                             8.0
Magnesium, Mg                      500.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Name: M 19-30, dtype: float64

## Linear Program Setup <a id='Linear_Program_Setup'></a>

### Create bmin and bmax  dataframes <a id='Bmin_Bmax'></a>

In [180]:
apikey = 'wkD3aDAMmafxVnYDrkUqEUA9aoUjGoGXoTviwdpc'

In [181]:
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
bmin.head()

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


In [182]:
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
bmax

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
"Sodium, Na",1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300


### Create the diet_min and diet_max dataframes <a id='Dietmin_Dietmax'></a>

In [183]:
!gpg --batch --passphrase "noodle octopus" -d ../students-9093fa174318.json.gpg > ../students-9093fa174318.json

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [184]:
user = "ligon"

serviceacct = {'ligon':'../students-9093fa174318.json'}
DRIs = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

# Define *minimums*
diet_min = read_sheets(DRIs,json_creds=serviceacct[user],sheet='diet_minimums').set_index('Nutrition')

# Define *maximums*
diet_max = read_sheets(DRIs,json_creds=serviceacct[user],sheet='diet_maximums').set_index('Nutrition')

### Create the df, Price, and FoodNutrients dataframes <a id='df_Price_FoodNutrients'></a>

#### Budget Friendly Diabetes Diet <a id='Budget1'></a>

In [185]:
budget_friendly_diabetes_diet_df = pd.read_csv("Budget_Friendly_Diabetes_Diet.csv")
display(budget_friendly_diabetes_diet_df.head())

,Food,Quantity,Units,Price,Date,Location,FDC Code
0,Potatoes,1.0,lb,$1.89,[2021-03-03 Wednesday],Berkeley Bowl West,1192521
1,Eggs,18.0,ct,$1.74,[2021-03-08 Sunday],Walmart,747997
2,Tofu,16.0,oz,$3.69,[2021-03-10 Wednesday],Berkeley Bowl West,496446
3,Pinto beans,8.0,lb,$5.98,[2021-03-10 Wednesday],Walmart,1249329
4,Low fat cottage cheese,16.0,oz,$3.29,[2021-03-10 Wednesday],Berkeley Bowl West,1191554


In [186]:
int_price_lst = [float(i[1:]) for i in budget_friendly_diabetes_diet_df["Price"]]
budget_friendly_diabetes_diet_df["Price"] = int_price_lst
budget_friendly_diabetes_diet_df = budget_friendly_diabetes_diet_df.rename(columns={"FDC Code":"FDC"})

In [187]:
# Convert food quantities to FDC units
budget_friendly_diabetes_diet_df['FDC Quantity'] = budget_friendly_diabetes_diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
budget_friendly_diabetes_diet_df['FDC Price'] = budget_friendly_diabetes_diet_df['Price']/budget_friendly_diabetes_diet_df['FDC Quantity']

budget_friendly_diabetes_diet_df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
budget_friendly_diabetes_diet_prices = budget_friendly_diabetes_diet_df.groupby('Food')['FDC Price'].min()

In [188]:

D = {}
count = 0
for food in budget_friendly_diabetes_diet_df.Food.tolist():
    try:
        FDC = budget_friendly_diabetes_diet_df.loc[budget_friendly_diabetes_diet_df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        print("Attribute Error")
        #warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

BudgetFoodNutrients = pd.DataFrame(D,dtype=float)

In [189]:
budget_p = budget_friendly_diabetes_diet_prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(budget_p.index.tolist()).intersection(BudgetFoodNutrients.columns.tolist()))
budget_p = budget_p[use]

In [190]:
display(budget_friendly_diabetes_diet_df.head())
display(BudgetFoodNutrients)
display(budget_p)

,Food,Quantity,Units,Price,Date,Location,FDC,FDC Quantity,FDC Price
0,Potatoes,1.0,lb,1.89,[2021-03-03 Wednesday],Berkeley Bowl West,1192521,4.535923700000001 hectogram,0.41667367552941853 / hectogram
1,Eggs,18.0,ct,1.74,[2021-03-08 Sunday],Walmart,747997,0.036000000000000004 hectogram,48.33333333333333 / hectogram
2,Tofu,16.0,oz,3.69,[2021-03-10 Wednesday],Berkeley Bowl West,496446,4.535923700000001 hectogram,0.8135057474621982 / hectogram
3,Pinto beans,8.0,lb,5.98,[2021-03-10 Wednesday],Walmart,1249329,36.287389600000004 hectogram,0.164795540983196 / hectogram
4,Low fat cottage cheese,16.0,oz,3.29,[2021-03-10 Wednesday],Berkeley Bowl West,1191554,4.535923700000001 hectogram,0.7253208425882471 / hectogram


,Potatoes,Eggs,Tofu,Pinto beans,Low fat cottage cheese,Canned tuna,Pita bread,Frozen fruit mix,Steel cut oats,Peanut butter,...,Greek Yogurt,Onions,Green beans (canned),Oranges,Brown Rice,Sweet Potato,Frozen vegetables,Oatmeal,Salsa,Popcorn (kernels no additives)
10:0,NaN,NaN,NaN,NaN,NaN,0.002,NaN,0.000,NaN,NaN,...,NaN,0.000,0.00,0.000,NaN,NaN,0.000,NaN,NaN,NaN
12:0,NaN,NaN,NaN,NaN,NaN,0.000,NaN,0.000,NaN,NaN,...,NaN,0.000,0.00,0.000,NaN,NaN,0.001,NaN,NaN,NaN
14:0,NaN,NaN,NaN,NaN,NaN,0.011,NaN,0.003,NaN,NaN,...,NaN,0.004,0.00,0.000,NaN,NaN,0.001,NaN,NaN,NaN
16:0,NaN,NaN,NaN,NaN,NaN,0.134,NaN,0.028,NaN,NaN,...,NaN,0.034,0.08,0.013,NaN,NaN,0.086,NaN,NaN,NaN
16:1,NaN,NaN,NaN,NaN,NaN,0.017,NaN,0.014,NaN,NaN,...,NaN,0.000,0.00,0.003,NaN,NaN,0.001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Vitamin E, added",NaN,NaN,NaN,NaN,NaN,0.000,NaN,0.000,NaN,NaN,...,NaN,0.000,0.00,0.000,NaN,NaN,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),NaN,NaN,NaN,NaN,NaN,0.200,NaN,2.600,NaN,NaN,...,NaN,0.400,38.90,0.000,NaN,NaN,NaN,NaN,NaN,NaN
Vitamins and Other Components,NaN,0.0,NaN,NaN,NaN,0.000,NaN,0.000,NaN,NaN,...,NaN,0.000,0.00,0.000,NaN,NaN,0.000,NaN,NaN,NaN
Water,NaN,86.3,NaN,NaN,NaN,79.000,NaN,87.590,NaN,NaN,...,NaN,89.110,93.60,86.750,NaN,NaN,82.080,NaN,NaN,NaN


Food
Greek Yogurt                                0.382502
Black Beans                                 0.220462
Pita bread                                  1.143464
Almonds                                     0.204589
Salsa                                       0.296301
Brown Rice                                  0.244272
Green beans (canned)                        0.287057
Spinach                                     1.452846
Tofu                                        0.813506
Whole-grain cereal (Honey Nut Cheerios)     1.937128
Oatmeal                                     0.325864
Potatoes                                    0.416674
Salad greens                                0.850984
Sweet Potato                                0.482812
Onions                                      0.135217
Canned tuna                                 1.466809
Popcorn (kernels no additives)              0.350388
Steel cut oats                              0.463265
Frozen vegetables                        

#### Dash Diet <a id='Dash1'></a>

In [191]:
dash_diet_df = pd.read_csv("Dash_Diet.csv")
display(dash_diet_df.head())

,Food,Quantity,Units,Price,Date,Location,FDC Code
0,Chicken breast,3.0,lb,$10.47,[2021-03-03 Wednesday],Safeway: 5100 Broadway,1098444
1,"Chicken drumsticks, dark meat",2.5,lb,$7.48,[2021-03-03 Wednesday],Safeway: 5100 Broadway,172855
2,Salmon,1.0,lb,$9.99,[2021-03-03 Wednesday],Safeway: 5100 Broadway,1266792
3,Tuna Canned,12.0,oz,$4.99,[2021-03-08 Sunday],Safeway: 5100 Broadway,1099042
4,Tuna Ahi Steak,12.0,oz,$6.99,[2021-03-08 Sunday],Safeway: 5100 Broadway,487435


In [192]:
int_price_lst = [float(i[1:]) for i in dash_diet_df["Price"]]
dash_diet_df["Price"] = int_price_lst
dash_diet_df = dash_diet_df.rename(columns={"FDC Code":"FDC"})

In [193]:
# Convert food quantities to FDC units
dash_diet_df['FDC Quantity'] = dash_diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
dash_diet_df['FDC Price'] = dash_diet_df['Price']/dash_diet_df['FDC Quantity']

dash_diet_df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
dash_diet_prices = dash_diet_df.groupby('Food')['FDC Price'].min()

In [194]:

D = {}
count = 0
for food in dash_diet_df.Food.tolist():
    try:
        FDC = dash_diet_df.loc[dash_diet_df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        print("Attribute Error")
        #warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

DashFoodNutrients = pd.DataFrame(D,dtype=float)

In [195]:
dash_p = dash_diet_prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(dash_p.index.tolist()).intersection(DashFoodNutrients.columns.tolist()))
dash_p = dash_p[use]

In [196]:
display(dash_diet_df.head())
display(DashFoodNutrients)
display(dash_p)

,Food,Quantity,Units,Price,Date,Location,FDC,FDC Quantity,FDC Price
0,Chicken breast,3.0,lb,10.47,[2021-03-03 Wednesday],Safeway: 5100 Broadway,1098444,13.6077711 hectogram,0.7694132950252227 / hectogram
1,"Chicken drumsticks, dark meat",2.5,lb,7.48,[2021-03-03 Wednesday],Safeway: 5100 Broadway,172855,11.339809250000002 hectogram,0.6596230884571537 / hectogram
2,Salmon,1.0,lb,9.99,[2021-03-03 Wednesday],Safeway: 5100 Broadway,1266792,4.535923700000001 hectogram,2.2024179992269266 / hectogram
3,Tuna Canned,12.0,oz,4.99,[2021-03-08 Sunday],Safeway: 5100 Broadway,1099042,3.401942775 hectogram,1.4668089177367187 / hectogram
4,Tuna Ahi Steak,12.0,oz,6.99,[2021-03-08 Sunday],Safeway: 5100 Broadway,487435,3.401942775 hectogram,2.054708283563059 / hectogram


,Chicken breast,"Chicken drumsticks, dark meat",Salmon,Tuna Canned,Tuna Ahi Steak,Ground Turkey,Lean Ground Beef,Pork Top Loin,Broccoli,Carrots,...,Lentils,Sweet peas,Olive oil,Low-fat mayonnaise,"Instant oatmeal, flavored",Whole wheat bagel,Romaine lettuce,Mushrooms,Corn (Canned),Corn
10:0,0.018,0.005,NaN,0.002,NaN,NaN,NaN,0.004,NaN,0.00,...,NaN,NaN,0.0,NaN,0.001,0.00,NaN,0.00,0.003,0.003
12:0,0.010,0.021,NaN,0.000,NaN,NaN,NaN,0.003,NaN,0.00,...,NaN,NaN,0.0,NaN,0.000,0.00,NaN,0.00,0.000,0.001
14:0,0.062,0.253,NaN,0.011,NaN,NaN,NaN,0.098,NaN,0.00,...,NaN,NaN,0.0,NaN,0.003,0.00,NaN,0.00,0.002,0.001
14:1,NaN,0.091,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15:0,NaN,0.027,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),0.500,0.400,NaN,0.200,NaN,NaN,NaN,0.000,NaN,13.20,...,NaN,NaN,60.2,NaN,0.400,1.50,NaN,0.00,3.400,0.300
Vitamins and Other Components,0.000,0.000,NaN,0.000,NaN,NaN,NaN,0.000,NaN,0.00,...,NaN,NaN,0.0,NaN,0.000,0.00,NaN,0.00,0.000,0.000
Water,63.160,44.910,NaN,79.000,NaN,NaN,NaN,70.140,NaN,88.29,...,NaN,NaN,0.0,NaN,75.230,37.55,NaN,92.45,79.040,76.050


Food
Egg                                     85.000000
Low-fat mayonnaise                       0.700776
Rasins                                   1.171879
Tomatoes                                 0.967829
Tuna Ahi Steak                           2.054708
Salmon                                   2.202418
Ground Turkey                            0.879644
Green Beans                              0.659182
Cantaloupe Melon                         0.659182
Walnuts                                  1.194905
Mushrooms                                1.318364
Lentils                                  0.216053
Chicken breast                           0.769413
Instant oatmeal, flavored               74.500000
Spinach                                  1.452846
Corn                                     0.152119
Kale                                     1.254745
Olive oil                                0.817526
Romaine lettuce                          0.593043
Yellow Peach                             1.84

### Create the Amin, Amax, and A dataframes <a id='Amin_Amax_A'></a>

#### Budget Friendly Diabetes Diet <a id='Budget2'></a>

In [197]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
BudgetAall = BudgetFoodNutrients[budget_p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
BudgetAmin = BudgetAall.loc[bmin.index]

BudgetAmax = BudgetAall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
BudgetA = pd.concat([BudgetAmin,-BudgetAmax])

BudgetA.head()

,Greek Yogurt,Black Beans,Pita bread,Almonds,Salsa,Brown Rice,Green beans (canned),Spinach,Tofu,Whole-grain cereal (Honey Nut Cheerios),...,Strawberries,Pinto beans,Canned salmon,Broccoli,Peanut butter,Frozen fruit mix,Eggs,Nut and fruit trail mix,Bananas,Oranges
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,467.00,234.00,231.00,604.00,48.0,378.00,21.00,24.00,82.00,376.00,...,32.00,229.0,137.00,34.0,594.0,44.0,231.0,452.0,89.00,47.00
Protein,3.33,19.15,10.26,20.23,0.0,8.89,1.04,2.35,10.59,8.85,...,0.67,20.0,20.55,2.7,25.0,0.7,10.7,10.9,1.09,0.94
"Fiber, total dietary",3.30,27.70,5.10,10.50,3.2,4.40,1.90,2.40,1.20,7.10,...,2.00,34.3,0.00,2.0,9.4,1.5,0.0,6.5,2.60,2.40
"Folate, DFE",0.00,0.00,0.00,53.00,0.0,0.00,23.00,0.00,0.00,1106.00,...,24.00,0.0,13.00,0.0,0.0,20.0,0.0,51.0,20.00,30.00
"Calcium, Ca",133.00,34.00,154.00,259.00,0.0,0.00,36.00,94.00,118.00,425.00,...,16.00,0.0,220.00,41.0,59.0,11.0,0.0,62.0,5.00,40.00


#### Dash Diet <a id='Dash2'></a>

In [198]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
DashAall = DashFoodNutrients[dash_p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
DashAmin = DashAall.loc[bmin.index]

DashAmax = DashAall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
DashA = pd.concat([DashAmin,-DashAmax])

DashA.head()

,Egg,Low-fat mayonnaise,Rasins,Tomatoes,Tuna Ahi Steak,Salmon,Ground Turkey,Green Beans,Cantaloupe Melon,Walnuts,...,Peanut butter,Whole wheat bagel,"Almonds, Low Sodium Lightly Salted","Milk, Fat-free",Green Peas,Collards,Tuna Canned,Bananas,Pork Top Loin,Oranges
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,231.0,400.0,464.00,14.0,106.00,193.00,143.00,31.00,34.00,667.00,...,594.0,250.0,604.00,348.00,82.00,32.00,90.0,89.00,696.00,47.00
Protein,10.7,0.0,14.29,0.0,23.89,19.33,19.64,1.83,0.82,16.67,...,25.0,10.2,20.23,34.78,5.88,3.02,19.0,1.09,21.34,0.94
"Fiber, total dietary",0.0,0.0,7.10,0.0,0.00,0.00,0.00,2.70,0.80,6.70,...,9.4,4.1,10.50,0.00,5.90,4.00,0.0,2.60,0.00,2.40
"Folate, DFE",0.0,0.0,0.00,0.0,0.00,0.00,0.00,33.00,14.00,0.00,...,0.0,108.0,53.00,0.00,0.00,129.00,4.0,20.00,0.00,30.00
"Calcium, Ca",0.0,7.0,71.00,0.0,18.00,13.00,0.00,37.00,9.00,97.00,...,59.0,105.0,259.00,1304.00,25.00,232.00,18.0,5.00,7.00,40.00


### Create the b dataframe <a id='b'></a>

In [199]:
b = pd.concat([bmin,-bmax]) # Note sign change for max constraints

b.head()

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


### Solving the Linear Program Problem <a id='Solved'></a>

#### Budget Friendly Diabetes Diet <a id='Budget3'></a>

In [200]:
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
group = 'M 19-30'

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

budget_result = lp(budget_p, -BudgetA, -b[group], method='interior-point')

budget_result

     con: array([], dtype=float64)
     fun: 2.6102339974778848
 message: 'Optimization terminated successfully.'
     nit: 16
   slack: array([4.13976267e+02, 3.31707052e+01, 4.36044814e+01, 2.12618086e+02,
       3.50271716e-06, 5.43195757e+01, 1.87183540e+01, 4.82890716e+02,
       4.12789619e+00, 8.65932473e+02, 1.21973662e-05, 2.89879254e+00,
       3.62919916e-09, 1.36707990e-06, 1.31447768e-08, 9.56348052e-02,
       2.65637667e-07, 5.27509864e+01, 1.50686844e-07, 4.82574839e+00,
       1.40551635e+03])
  status: 0
 success: True
       x: array([1.05125883e-09, 2.90730765e-09, 3.82873886e-10, 2.87484253e+00,
       1.80648328e-08, 3.34405964e-09, 1.31116777e-08, 1.71481604e-01,
       4.12541521e-10, 3.20000001e-01, 1.03389216e-09, 1.23631784e-09,
       1.39042818e-01, 8.47535071e-10, 6.57456068e-09, 3.20901604e-10,
       1.04653445e-09, 1.78824665e-09, 2.27969277e+00, 1.13070909e-09,
       5.56622875e-10, 7.09175134e-10, 9.51228981e-01, 3.25960635e-10,
       1.24146772e-09

In [201]:
print("Cost of diet for %s is $%4.2f per day." % (group,budget_result.fun))

Cost of diet for M 19-30 is $2.61 per day.


In [202]:
# Put back into nice series
budget_diet = pd.Series(budget_result.x,index=budget_p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(budget_diet[budget_diet >= tol])  # Drop items with quantities less than precision of calculation.


You'll be eating (in 100s of grams or milliliters):
Food
Almonds                                    2.874843
Spinach                                    0.171482
Whole-grain cereal (Honey Nut Cheerios)    0.320000
Salad greens                               0.139043
Frozen vegetables                          2.279693
Pinto beans                                0.951229
Oranges                                    0.997706
dtype: float64


In [203]:
budget_tab = pd.DataFrame({"Outcome":np.abs(BudgetA).dot(budget_diet),"Recommendation":np.abs(b[group])})
print("\nWith the following nutritional outcomes of interest:")
budget_tab


With the following nutritional outcomes of interest:


,Outcome,Recommendation
Nutrition,,
Energy,2813.976267,2400.0
Protein,89.170705,56.0
"Fiber, total dietary",77.204481,33.6
"Folate, DFE",612.618086,400.0
"Calcium, Ca",1000.000004,1000.0
"Carbohydrate, by difference",184.319576,130.0
"Iron, Fe",26.718354,8.0
"Magnesium, Mg",882.890716,400.0
Niacin,20.127896,16.0


In [204]:
print("\nConstraining nutrients are:")
budget_excess = budget_tab.diff(axis=1).iloc[:,1]
print(budget_excess.loc[np.abs(budget_excess) < tol].index.tolist())


Constraining nutrients are:
['Thiamin', 'Vitamin B-12', 'Vitamin C, total ascorbic acid', 'Vitamin K (phylloquinone)']


#### Dash Diet <a id='Dash3'></a>

In [205]:
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
group = 'M 19-30'

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

dash_result = lp(dash_p, -DashA, -b[group], method='interior-point')

dash_result

     con: array([], dtype=float64)
     fun: 3.9649765626549276
 message: 'Optimization terminated successfully.'
     nit: 16
   slack: array([1.32744933e+02, 5.48904585e+01, 3.11148006e+01, 9.02957489e+01,
       2.66080747e+02, 8.22242183e+01, 1.17145036e+01, 1.86364485e+02,
       6.72979539e-09, 1.09076128e+03, 3.92261427e-07, 3.17577713e-01,
       1.32459353e-01, 5.70377779e-07, 1.77662773e-10, 1.77603709e-01,
       2.43902889e-08, 2.30702391e-09, 3.11264998e-07, 6.74273082e-10,
       2.85322130e-07])
  status: 0
 success: True
       x: array([1.59612001e-12, 1.38908077e-10, 7.48374722e-11, 1.25880305e-10,
       5.27143254e-11, 4.33033954e-11, 1.07041296e-10, 1.95825498e-10,
       2.02765344e-10, 7.72498288e-11, 1.02749033e-10, 1.25581842e-09,
       1.15960652e-10, 1.89453435e-12, 8.02218906e-11, 7.06684925e-11,
       8.43415560e-11, 4.67736688e-10, 1.70816399e-10, 6.60209589e-11,
       4.21919939e-10, 3.10414371e-10, 7.51747249e-10, 1.39846957e-10,
       5.32708873e-10

In [206]:
print("Cost of diet for %s is $%4.2f per day." % (group,dash_result.fun))

Cost of diet for M 19-30 is $3.96 per day.


In [207]:
# Put back into nice series
dash_diet = pd.Series(dash_result.x,index=dash_p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(dash_diet[dash_diet >= tol])  # Drop items with quantities less than precision of calculation.


You'll be eating (in 100s of grams or milliliters):
Food
Low-fat mozzarella cheese             0.407218
Lima Beans                            0.507926
Kidney beans (Canned)                 5.540602
Broccoli                              0.789451
Carrots                               0.951069
Almonds, Low Sodium Lightly Salted    0.563153
Milk, Fat-free                        0.309527
Collards                              0.146101
Tuna Canned                           0.788077
dtype: float64


In [208]:
dash_tab = pd.DataFrame({"Outcome":np.abs(DashA).dot(dash_diet),"Recommendation":np.abs(b[group])})
print("\nWith the following nutritional outcomes of interest:")
dash_tab


With the following nutritional outcomes of interest:


,Outcome,Recommendation
Nutrition,,
Energy,2532.744933,2400.0
Protein,110.890459,56.0
"Fiber, total dietary",64.714801,33.6
"Folate, DFE",490.295749,400.0
"Calcium, Ca",1266.080747,1000.0
"Carbohydrate, by difference",212.224218,130.0
"Iron, Fe",19.714504,8.0
"Magnesium, Mg",586.364485,400.0
Niacin,16.000000,16.0


In [209]:
print("\nConstraining nutrients are:")
dash_excess = dash_tab.diff(axis=1).iloc[:,1]
print(dash_excess.loc[np.abs(dash_excess) < tol].index.tolist())


Constraining nutrients are:
['Niacin', 'Potassium, K', 'Vitamin A, RAE', 'Vitamin B-12', 'Vitamin C, total ascorbic acid', 'Vitamin E (alpha-tocopherol)', 'Vitamin K (phylloquinone)', 'Zinc, Zn', 'Sodium, Na']


## Solve the Subsistence Problem <a id='Subsistence'></a>

### Create the solve_subsistence_problem function <a id='Subsistence_function'></a>

In [210]:
from  scipy.optimize import linprog as lp
import numpy as np

def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[diet_min.index]

    Amax = Aall.loc[diet_max.index]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,-Amax])

    b = pd.concat([diet_min,-diet_max]) # Note sign change for max constraints

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

### Results For Target Group <a id='Results'></a>

#### Budget Friendly Diabetes Diet <a id='Budget4'></a>

In [211]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $7.03 per day.


Diet (in 100s of grams or milliliters):
Food
Navy Beans      0.539075
Cabbage         0.997122
Spinach         7.315484
Liver (Pork)    0.902305
Milk (Whole)    1.628001
Wheat Cereal    0.313215
Sugar           0.455579
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000003          2400.0
Protein                           60.402307            56.0
Fiber, total dietary              33.600000            33.6
Folate, DFE                     1653.368775           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      148.346580           130.0
Iron, Fe                          47.590071             8.0
Magnesium, Mg                    624.609875           400.0
Niacin                            25.136346            16.0
Phosp

In [212]:
group = 'M 19-30'
tol = 1e-6

budget_result = solve_subsistence_problem(BudgetFoodNutrients,budget_friendly_diabetes_diet_prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,budget_result.fun))

# Put back into nice series
budget_diet = budget_result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(budget_diet[budget_diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

budget_tab = pd.DataFrame({"Outcome":np.abs(budget_result.A).dot(budget_diet),"Recommendation":np.abs(budget_result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(budget_tab)
print()

print("\nConstraining nutrients are:")
budget_excess = budget_tab.diff(axis=1).iloc[:,1]
print(budget_excess.loc[np.abs(budget_excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $2.61 per day.


Diet (in 100s of grams or milliliters):
Food
Almonds                                    2.874843
Spinach                                    0.171482
Whole-grain cereal (Honey Nut Cheerios)    0.320000
Salad greens                               0.139043
Frozen vegetables                          2.279693
Pinto beans                                0.951229
Oranges                                    0.997706
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2813.976267          2400.0
Protein                           89.170705            56.0
Fiber, total dietary              77.204481            33.6
Folate, DFE                      612.618086           400.0
Calcium, Ca                     1000.000004          1000.0
Carbohydrate, by difference      184.319576           13

#### Dash Diet <a id='Dash4'></a>

In [213]:
group = 'M 19-30'
tol = 1e-6

dash_result = solve_subsistence_problem(DashFoodNutrients,dash_diet_prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,dash_result.fun))

# Put back into nice series
dash_diet = dash_result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(dash_diet[dash_diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

dash_tab = pd.DataFrame({"Outcome":np.abs(dash_result.A).dot(dash_diet),"Recommendation":np.abs(dash_result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(dash_tab)
print()

print("\nConstraining nutrients are:")
dash_excess = dash_tab.diff(axis=1).iloc[:,1]
print(dash_excess.loc[np.abs(dash_excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $3.96 per day.


Diet (in 100s of grams or milliliters):
Food
Low-fat mozzarella cheese             0.407218
Lima Beans                            0.507926
Kidney beans (Canned)                 5.540602
Broccoli                              0.789451
Carrots                               0.951069
Almonds, Low Sodium Lightly Salted    0.563153
Milk, Fat-free                        0.309527
Collards                              0.146101
Tuna Canned                           0.788077
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2532.744933          2400.0
Protein                          110.890459            56.0
Fiber, total dietary              64.714801            33.6
Folate, DFE                      490.295749           400.0
Calcium, Ca                     1266.080747          1000

### Plotting Foods vs Prices <a id='Plotting'></a>

#### Budget Friendly Diabetes Diet <a id='Budget5'></a>

In [214]:
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Almonds'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = solve_subsistence_problem(BudgetFoodNutrients,budget_friendly_diabetes_diet_prices,diet_min[group],diet_max[group],tol=tol)

my_p = budget_friendly_diabetes_diet_prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = budget_friendly_diabetes_diet_prices[ReferenceGood]*s
    result = solve_subsistence_problem(BudgetFoodNutrients,my_p,diet_min[group],diet_max[group],max_weight=12,tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

#### Dash Diet <a id='Dash5'></a>

In [215]:
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Kidney beans (Canned)'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = solve_subsistence_problem(DashFoodNutrients,dash_diet_prices,diet_min[group],diet_max[group],tol=tol)

my_p = dash_diet_prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = dash_diet_prices[ReferenceGood]*s
    result = solve_subsistence_problem(DashFoodNutrients,my_p,diet_min[group],diet_max[group],max_weight=12,tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')